In [ ]:
src_path = r"vector\\"
target_path = "output\\"
trim_files(src_path,target_path)

In [ ]:
# 查看各种语句的几个例子

object_func_call  = set()
folder_path = '.\output\\'
file_list = read_dir(folder_path)
function_list = set()
for filename in file_list:
    with open(folder_path + filename, 'r') as f:
        for line in f:
            if is_func_call(line):
                res = parse_func_call(line)
                function_list.add(res[0])
                
print(function_list)

In [1]:
from tool_function import *
from graph import *

# 使用解析好的资源构建链表或者图
# 首先是设计图的节点类的结构，然后是边的关系，最后是图的结构
# 读取一个文件，解析每一行，判断每一行语句的种类，然后进行处理
# 1. 读取文件
# 2. 解析每一行
# 3. 判断语句种类
# 4. 处理语句

def read_to_graph(filename,graph,nodelist,mapstack):
    line_pos = 0
    with open(filename, "r") as f:
        for line in f:
            if is_comment(line):
                pass
            elif is_empty(line):
                pass
            elif is_other(line):
                res = parse_statement(line) # 里面又None
                try:
                    # res[1]是变量名，res[3]是变量值
                    # res[3]这个变量值可能引用到了其他变量，所以需要进行解析
                    node = Node(res[1],[filename,line_pos])
                    graph.add_vertex(node)
                    # res[3]引用的都是(a,b,c)这种，里面没有括号，有括号的没有引用
                    # 姑且认为包括括号的都没有引用其他变量   
                    if "=" in res[3]:
                        pass
                    else:
                        val_list = res[3].split(",")
                        for val in val_list:
                            if(is_macro(val.strip())): # 去除空格
                                pass
                                #print(val,end=" ")
                            elif(is_decimal_or_hex(val.strip())):
                                pass
                                # print(val,end=" ")
                            elif(is_valid_variable(val.strip())):
                                val_node = Node(val.strip(),[filename,line_pos])
                                # a的链表不需要本次的引用，res[1]才需要本行，a的其他地方不需要本行。
                                # graph.add_vertex(val_node)
                                graph.add_edge(node,val_node)
                    
                except:
                    node = Node("NULL",[filename,line_pos])
                    nodelist.add_node(node)
            elif is_object_func_call(line):
                res = parse_object_func_call(line) # 解析对象调用
                if res[1] in create_func:
                    # 如果在创建函数中，会根据已有的资源创建一个对象，然后连接起他们
                    ext_node_list = [] # 已经存在的节点
                    dontext_node_list = [] # 不存在的节点
                    for i in res[2]:
                        p_v = parse_func_assignment(i) #parameters and values
                        if(is_macro(p_v[1])):
                            pass
                        elif(is_decimal_or_hex(p_v[1])):
                            pass
                        elif(is_valid_variable(p_v[1])):
                            # 如果是一个合法的变量，那么就是一个资源，创建一个资源节点
                            node = Node(p_v[1],[filename,line_pos])
                            if not graph.has_vertex(node):
                                graph.add_vertex(node)
                                dontext_node_list.append(node)
                            else:
                                ext_node_list.append(node)


                    # 将存在的与不存在的进行连接
                    for ex in ext_node_list:
                        for dont in dontext_node_list:
                            # 比如用pBuf_13创建了pBuf0_14，最后文件中pBuf_14依赖于pBuf_13，
                            # 访问pBuf_14的时候，需要去访问pBuf_13，所以是pBuf_14->pBuf_13
                            graph.add_edge(dont,ex)

                elif res[1] in set_func:
                    # TODO：如果在设置函数，将已有的一个资源设置为另一个资源的属性
                    # 一般是前面几个参数是资源，最后一个参数是属性
                    node_list = []
                    for i in res[2]:
                        p_v = parse_func_assignment(i) #parameters and values
                        if(is_macro(p_v[1])):
                            pass
                        elif(is_decimal_or_hex(p_v[1])):
                            pass
                        elif(is_valid_variable(p_v[1])):
                            node = Node(p_v[1],[filename,line_pos])
                            graph.add_vertex(node)
                    # 将前面的资源与最后一个资源进行连接
                    for i in range(len(node_list)-2):
                        graph.add_edge(node_list[len(node_list)-1],node_list[i])

                elif res[1] in map_func:    
                    # TODO: map和unmap必须成对存在，他们的特点是第一个resource一样
                    # 他们最后都需要连接到创建的资源上
                    if res[1] =="Map":
                        ext_node_list = [] # 已经存在的节点，其实只有一个
                        dontext_node_list = [] # 不存在的节点，也只有以一个
                        for i in res[2]:
                            p_v = parse_func_assignment(i) #parameters and values
                            if(is_macro(p_v[1])):
                                pass
                            elif(is_decimal_or_hex(p_v[1])):
                                pass
                            elif(is_valid_variable(p_v[1])):
                                
                                # map的资源不需要加入到vertex中
                                node = Node(p_v[1],[filename,line_pos])
                                
                                if not graph.has_vertex(node):
                                    graph.add_vertex(node)
                                    dontext_node_list.append(node)
                                else:
                                    ext_node_list.append(node)
                        mapstack.push((ext_node_list[0],dontext_node_list[0]))
                    elif res[1]=="Unmap":
                        p_v = parse_func_assignment(res[2][0])
                        if(is_macro(p_v[1])):
                            pass
                        elif(is_decimal_or_hex(p_v[1])):
                            pass
                        elif(is_valid_variable(p_v[1])):
                             
                            if mapstack.top()[0].id == p_v[1]:
                                node = Node(p_v[1],[filename,line_pos])
                                graph.add_edge(mapstack.top()[1],mapstack.top()[0])
                                graph.add_edge(mapstack.top()[1],node)
                               
                                mapstack.pop()

                elif res[1] in other_func:
                    # TODO：如果在其他的函数中，保证加入到图中
                    node = Node("NULL",[filename,line_pos])
                    nodelist.add_node(node)
            elif is_func_call(line):
                # TODO: 默认保存
                node = Node("NULL",[filename,line_pos])
                nodelist.add_node(node)      
            line_pos = line_pos+1




In [2]:
graph = Graph()
nodelist = NodeList()
mapstack = MapStack()
folder_path = '.\output\\'
file_list = read_dir(folder_path)
for filename in file_list:
    read_to_graph(folder_path+filename,graph,nodelist,mapstack)
# graph.print_edges()

node = Node("pBuf_13",[])
print(graph.has_vertex(node))

True


In [3]:
res = []
graph.dfs("pMapRes_3200",res)
for i in  res:
    print(i.id,i.line_pos)

['pBuf_46', 'pBuf_46']
['pBuf_46', 'bufDesc_47']
pMapRes_3200 ['.\\output\\3DMarkICFWorkload_41.sdx', 353]
pBuf_46 ['.\\output\\3DMarkICFWorkload_41.sdx', 353]
pBuf_46 ['.\\output\\3DMarkICFWorkload_41.sdx', 355]
pBuf_46 ['.\\output\\3DMarkICFWorkload_5.sdx', 27]
bufDesc_47 ['.\\output\\3DMarkICFWorkload_5.sdx', 27]
bufDesc_47 ['.\\output\\3DMarkICFWorkload_5.sdx', 26]


In [ ]:
for i in graph.get_neighbors("pMapRes_3200"):
    print(i.id,i.line_pos)

In [ ]:
# 现在是做帧的简化步骤，要求是从第i帧到第j帧，从某个文件开始到某个文件结束，比如截取58到60帧，从文件序号59开始，到文件序号61结束
# 然后组成新的文件

# 1. 读取文件夹下的所有文件，然后按照文件名排序，然后按照文件名的数字排序
# 2. 先用所有的文件构成一个图，

# 建图后该怎么进行重构，
# 先试着截取1帧
new_file_list = [] # 保存文件行所在位置
folder_path = '.\output\\'
filename = "3DMarkICFWorkload_58.sdx"
with open(folder_path + filename, 'r') as f:
    line_pos = 0
    for line in f:
        new_file_list.append([filename,line_pos])
        if is_comment(line):
            pass
        elif is_empty(line):
            pass
        elif is_other(line):
            print(parse_statement(line))
            res = parse_statement(line)
            for i in range(3,len(res)-1):
                # 第一个是类型，第二个是变量名称，第三个是数组，第四个才是依赖
                if( is_decimal_or_hex(res[i])):
                    pass
                elif(is_macro(res[i])):
                    pass
                elif(is_valid_variable(res[i])):
                    # 当是合法的变量时候，应该去以res[i]去深度遍历所有依赖的节点。
                    
                else:
                    pass
        elif is_object_func_call(line):
            pass
        elif is_func_call(line):
            pass
    
        # 如果这行引用到了其他文件的变量，则去graph寻找
        
    line_pos = line_pos+1